In [3]:
import os

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random




# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers

% reload_ext autoreload
% autoreload 2

In [ ]:
TAGS = ["@nome_interessado_maiusculas@", "@endereco_destinatario@", "@bairro_destinatario@", "@cep_destinatario@", "@cidade_destinatario@", "@validade@"]
VALUES = ["Nome", "Logradouro", ["Número", "Complemento", "Bairro"], "Cep", "Cidade", "Validade"]

In [4]:
numero = "53504.006801/2018-10"

In [5]:
gc = functions.authenticate_google("files/anatel.json")
wb = gc.open(title=numero)

In [6]:
sht = wb.get_worksheet(0)

In [7]:
df = gs.get_as_dataframe(sht, dtype=str)[:18]

In [8]:
df["Complemento"] = df["Complemento"].str.replace("nan", "")

df.drop_duplicates(inplace=True)

df.head()

,Cpf,Nome,Fistel,Validade,Filtro,End_Sede,End_Corresp,Situação,Devedor,Cpf_RF,...,Situação_Cadastral,Fone,Logradouro,Num,Complemento,Bairro,Cidade,UF,Cep,Endereço_Completo
0,63655110510,AILTON ALMEIDA CORREIA,80106323504,21/09/2017,=B2=K2,"RUA SEDAN, 230\n- UTINGA\nCEP: 09230180\nAdama...",RUA TENENTE SILVIO FLEMING 11 APT 33 VILA BONI...,Entidade devedora (Bloqueada),Sim,636.551.105-10,...,REGULAR,99395905,RUA IRMAOS GRIFFO,116,ACACIA,POSTO DA MATA,NOVA VICOSA,BA,45928-000,RUA IRMAOS GRIFFO 116 ACACIA POSTO DA MATA \n ...
1,6449219866,CARLOS CESAR DOS SANTOS,80106205560,27/09/2017,=B3=K3,"Rua Miguel Ticotostes, 98, Casa\n- Dom Romeu\n...",RUA MADEIRA 76 APTO 3 CANINDE \n CEP: 03033-0...,Entidade devedora (Bloqueada),Sim,064.492.198-66,...,REGULAR,36626674,OUTROS MIGUEL TICOTOSTI,98,,DOM ROMEU ALBERTO,BATATAIS,SP,14300-000,OUTROS MIGUEL TICOTOSTI 98 DOM ROMEU ALBERTO ...
2,16523980820,CARLOS EDUARDO GONCALVES DA SILVA,80106333720,28/09/2017,=B4=K4,"RUA DAS PALMAS, 223\n- VILA SAO JOSE\nCEP: 099...",RUA MANOEL DOS SANTOS 321 PONTILHAO \n CEP: ...,Entidade não possui débitos,Não,165.239.808-20,...,REGULAR,40775213,RUA DAS PALMAS,223,,VILA SAO JOSE,DIADEMA,SP,09950-510,RUA DAS PALMAS 223 VILA SAO JOSE \n CEP: 0995...
3,01270574809,CLAUDINEI CANDIDO DA SILVA,02035683971,23/09/2017,=B5=K5,"RUA.JOAO VELHO DO REGO N.1222, .\n- SAO MATHEU...",RUA DOZE 4 PQ MA DOMITILA \n CEP: 04856-000 ...,Entidade devedora (Bloqueada),Sim,012.705.748-09,...,REGULAR,61123149,RUA JOAO VELHO DO REGO,1222,,SAO MATEUS,SAO PAULO,SP,03967-000,RUA JOAO VELHO DO REGO 1222 SAO MATEUS \n CEP...
4,02795241889,CLAUDIO DOMINGOS VENANCIO,80106237330,27/09/2017,=B6=K6,"RUA WALMOR URBAN, 1621, casa\n- VILA REDENCAO\...",RUA ADMAR LEOPOLDO GHELARDI 2-135 NOBUJE NAGA...,Entidade devedora (Bloqueada),Sim,027.952.418-89,...,REGULAR,35625128,RUA JOSE PERES,913,,JD BRASILIA,PIRASSUNUNGA,SP,13631-656,RUA JOSE PERES 913 JD BRASILIA \n CEP: 13631-...


In [30]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [31]:
browser = sei.login_sei(browser, usr, pwd)

In [32]:
sei_ = sei.Sei(browser)

sei_.itera_processos()

In [33]:
p = sei_.go_to_processo(numero)

In [21]:
for i in df.iterrows():
    
    dados = i[1].to_dict()    
    
    tag = sei_.atualizar_contato(dados['Nome'], dados)

In [34]:
tipo = "RC_Oficio de Cassação"

In [35]:
def string_endereço(row):
    
    s = 'À<br>' + str(row[1]["Nome"]).title() + '<br>' + str(row[1]["Logradouro"]).title() + ", " + str(row[1]["Num"]) \
         + " " + str(row[1]["Complemento"]).title() + " " + str(row[1]["Bairro"]).title() + "<br>" +  "CEP: " \
         + str(row[1]["Cep"]) +" - " + str(row[1]["Cidade"]).title() + " - " + str(row[1]["UF"]) + "<br><br>" + "<b>FISTEL: " \
         + str(row[1]["Fistel"]) + "</b>"
            
    return s

In [36]:
for row in df.iterrows():    
    
    dados = {'À': string_endereço(row)}    
    
    clip.copy(dados["À"])
    
    p.incluir_oficio(tipo, dados)
        
    sleep(random.randint(60,120))

In [ ]:
for p in sei_.get_processos().values():
    
    if p["aviso"] and p["tipo"] == "Outorga: Rádio do Cidadão":
        
        p = sei_.go_to_processo(p["numero"])
        
        teste = input()
        
        if teste == 'y':
            
            p.edita_marcador()
            p.edita_postit()
            
            p.get_acoes()['conc']

In [ ]:
p.edita_marcador("RC")

In [ ]:
select = Select(p.wait_for_element(helpers.Marcador.LISTA_MARCADORES))

In [27]:
windows = sei_.driver.window_handles

sei_.driver.switch_to_window(windows[0])

sei_.driver.switch_to.default_content()

NoAlertPresentException: Message: No modal dialog is currently open
